# Pandas File Format Benchmarking

## Setup and collect benchmark results

In [ ]:
import os
import numpy as np
import pandas as pd

experiment_name = "TODO"
number_of_repeats = 10  # How often to repeat benchmarks?
output_dir = "output"
output_file_type = "png"  # E.g. pdf of png
test_data_path = ""

In [ ]:
# Read data
if test_data_path:
    print("Reading data...")
    DF = pd.read_csv(test_data_path)
else:
    # Generate data
    print("Generating data...")
    np.random.seed = 42
    DF_SIZE = 1000_000
    DF = pd.DataFrame({
        'a': np.random.rand(DF_SIZE),
        'b': np.random.rand(DF_SIZE),
        'c': np.random.rand(DF_SIZE),
        'd': np.random.rand(DF_SIZE),
        'e': np.random.rand(DF_SIZE)
    })

In [ ]:
# Calculate and collect results
from format_benchmark_tool.format_benchmark_tool import FormatBenchmarkTool
benchmarking = FormatBenchmarkTool(DF, number_of_repeats=number_of_repeats)
memory_usage: float = benchmarking.get_memory_usage()
results = pd.DataFrame(benchmarking.get_results())

In [ ]:
# Save the results
results.to_csv(os.path.join(output_dir, f"results_{experiment_name}.csv"))

## Analyze results and draw plots

In [ ]:
# Setup
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(palette="colorblind")

In [ ]:
# Filter for a selection of formats
selected_formats = ['csv', 'json', 'xml', 'excel', 'pickle', 'hdf5', 'feather', 'parquet', 'orc', 'stata']
selected_idx = results['format'] == selected_formats[0]
for format in selected_formats[1:]:
    selected_idx += results['format'] == format
results = results[selected_idx]

In [ ]:
# Prepare time data
time_data = pd.DataFrame([], columns=['format', 'time', 'type'])

write_times = results[['format', 'write_time']].copy()
write_times.rename(columns={'write_time': 'time'}, inplace=True)
write_times['type'] = 'Write time'

read_times = results[['format', 'read_time']].copy()
read_times.rename(columns={'read_time': 'time'}, inplace=True)
read_times['type'] = 'Read time'

time_data = pd.concat([time_data, write_times, read_times], ignore_index=True)

# Plot minimum write and read times (with std deviation) per file format as barplot
ax = sns.barplot(data=time_data, x="time", y="format", hue='type', estimator='min', errorbar="ci")
ax.set_title(f"Minimum write and read times per file format\nwith confidence intervals; {number_of_repeats} trials")
ax.xaxis.set_label_text("Minimum time [s]")
ax.yaxis.set_label_text("File format")
plt.legend(title='Method')

for p in ax.patches:
    ax.annotate(
        str(p.get_width().round(5)), 
        (p.get_x() + p.get_width(), p.get_y() + p.get_height() / 2), 
        ha = 'left', va = 'center', 
        size=10,
        xytext = (10, 0), 
        textcoords = 'offset points')

plt.savefig(os.path.join(output_dir, f"time_{experiment_name}.{output_file_type}"))
plt.show()

In [ ]:
# Plot file sizes
tmp = results[['format', 'file_size']].copy()

# Add in in-memory usage as the first entry as a reference point
tmp = pd.concat([pd.DataFrame([['in-memory', memory_usage]], columns=['format', 'file_size']), tmp], ignore_index=True)

# Convert to megabytes
tmp['file_size_mb'] = tmp['file_size'] / 1024 / 1024

ax = sns.pointplot(data=tmp, x='file_size_mb', y='format', errorbar='ci', join=False)
ax.set_title("Size of output file per file format")
ax.xaxis.set_label_text("Output file size [MB]")
ax.yaxis.set_label_text("File format")

plt.savefig(os.path.join(output_dir, f"size_{experiment_name}.{output_file_type}"))
plt.show()

In [ ]:
# Barplot showing the format's file size as a factor compared to in-memory usage
# (e.g. 1.5 means the file is 1.5 times larger than the in-memory usage)
tmp['factor'] = tmp['file_size'] / memory_usage

# Colors representing the size factor (green = small, red = large)
# This has to be done on the mean for each format, as the plot uses the mean estimator
colors = []
for format in tmp['format'].unique():
    size_factor = tmp[tmp['format'] == format]['factor'].mean()
    if size_factor < 1:
        colors.append('green')
    elif size_factor > 1:
        colors.append('red')
    else:
        colors.append('black')

ax = sns.barplot(data=tmp, x='factor', y='format', palette=colors)
ax.set_title(f"Size of output file as a factor compared to in-memory usage of {memory_usage/1024/1024:.2f} bytes")
ax.xaxis.set_label_text("Output file size as a factor of in-memory usage")
ax.yaxis.set_label_text("File format")

for p in ax.patches:
    ax.annotate(
        str(p.get_width().round(5)), 
        (p.get_x() + p.get_width(), p.get_y() + p.get_height() / 2),
        ha = 'left', va = 'center', 
        size=10,
        xytext = (10, 0), 
        textcoords = 'offset points')
    
plt.savefig(os.path.join(output_dir, f"size_factor_{experiment_name}.{output_file_type}"))
plt.show()